In [1]:
using StructuralIdentifiability

In [2]:
using HomotopyContinuation

In [3]:
using AbstractAlgebra

In [4]:
using SymPy

In [5]:
include("functions.jl")

nemo2hc (generic function with 1 method)

In [6]:
three_compartments = @ODEmodel(
    x1'(t) = - (k01 + k21 + k31)* x1(t) + k12 * x2(t) + u(t),
    x2'(t) = k21 * x1(t) - (k12 + k32) * x2(t) + k23 * x3(t),
    x3'(t) = k31 * x1(t) + k32 * x2(t) - (k23 + k03) * x3(t),
    y1(t) = x2(t),
    y2(t) = x3(t)
)

[ Info: Summary of the model:
[ Info: State variables: x1, x2, x3
[ Info: Parameters: k01, k31, k21, k32, k12, k23, k03
[ Info: Inputs: u
[ Info: Outputs: y1, y2


x1'(t) = -k01*x1(t) - k31*x1(t) - k21*x1(t) + k12*x2(t) + u(t)
x2'(t) = k21*x1(t) - k32*x2(t) - k12*x2(t) + k23*x3(t)
x3'(t) = k31*x1(t) + k32*x2(t) - k23*x3(t) - x3(t)*k03
y1(t) = x2(t)
y2(t) = x3(t)


# Numerical solver

In [ ]:
function nemo2hc(expr_tree::Union{Expr,Symbol, Int64})
    #traverse expr_tree
	if typeof(expr_tree) == Int64
        return Expression(expr_tree)
    end
    if typeof(expr_tree) == Symbol
        return HomotopyContinuation.variables(expr_tree)[1]
    end
    if typeof(expr_tree) == Expr
        if expr_tree.head == :call
            if expr_tree.args[1] in [:+, :-, :*, :/, :^]
                return reduce(eval(expr_tree.args[1]), map(nemo2hc, expr_tree.args[2:end]))
            end
        end
    end
end

In [ ]:
function create_system_of_equations_num_hc(id_funct, names, initial_conditions)
    id_funct_num_hc = Array{Expression}(undef, 0)
    id_funct_den_hc = Array{Expression}(undef, 0)
    for el in id_funct
        num = numerator(el)
        den = denominator(el)
        push!(id_funct_num_hc, nemo2hc(Meta.parse(string(num))))
        push!(id_funct_den_hc, nemo2hc(Meta.parse(string(den))))
    end
	 
    # Substitute variable values into the polynomials
    id_funct_num_hc_transf = [HomotopyContinuation.subs(p, names => initial_conditions) for p in id_funct_num_hc]
    id_funct_den_hc_transf = [HomotopyContinuation.subs(p, names => initial_conditions) for p in id_funct_den_hc]
	
    # Create systems of equations
    lhs = id_funct_num_hc .* id_funct_den_hc_transf
    rhs = id_funct_num_hc_transf .* id_funct_den_hc
    system = lhs .- rhs

    return system
    
end

In [ ]:
function find_numerical_transformations_hc(model, names, initial_conditions, with_states = false)
    id_funct = find_identifiable_functions(model , with_states = with_states)
    for var in names
        @var names_map[1]
    end
    @var u
	varnames= Array{Variable}(undef, 0)
	for name in names
		push!(varnames, Variable(Meta.parse(name)))
	end
	#push!(varnames, u)
 
    system = create_system_of_equations_num_hc(id_funct,varnames, initial_conditions) 
    return results(HomotopyContinuation.solve(system), only_real = true)
end

In [128]:
Treatment = @ODEmodel(
	S'(t) = (-b*Inf(t)*S(t) - b*S(t)*d*Tr(t))//N(t),
	Inf'(t) = (b*Inf(t)*S(t) + b*S(t)*d*Tr(t) - Inf(t)*N(t)*g - Inf(t)*N(t)*a)//N(t),
	Tr'(t) = Inf(t)*g - nu*Tr(t),
	N'(t) = 0,
	y1(t) = Tr(t),
	y2(t) = N(t)
)

[ Info: Summary of the model:
[ Info: State variables: S, Inf, Tr, N
[ Info: Parameters: b, nu, d, g, a
[ Info: Inputs: 
[ Info: Outputs: y1, y2


S'(t) = (-b*S(t)*Inf(t) - b*S(t)*d*Tr(t))//N(t)
Inf'(t) = (b*S(t)*Inf(t) + b*S(t)*d*Tr(t) - Inf(t)*N(t)*g - Inf(t)*N(t)*a)//N(t)
Tr'(t) = -nu*Tr(t) + Inf(t)*g
N'(t) = 0
y1(t) = Tr(t)
y2(t) = N(t)


In [129]:
names_map = [("N", "N2"), ("S", "S2"), ("Tr", "Tr2"), ("Inf", "Inf2"), ("a", "a2"), ("b","b2"), ("g", "g2"), ("d", "d2"), ("nu", "nu2")]

9-element Vector{Tuple{String, String}}:
 ("N", "N2")
 ("S", "S2")
 ("Tr", "Tr2")
 ("Inf", "Inf2")
 ("a", "a2")
 ("b", "b2")
 ("g", "g2")
 ("d", "d2")
 ("nu", "nu2")

In [130]:
names = [el[1] for el in names_map]

9-element Vector{String}:
 "N"
 "S"
 "Tr"
 "Inf"
 "a"
 "b"
 "g"
 "d"
 "nu"

In [134]:
names = ["x1", "x2", "x3", "k01", "k03", "k12", "k21", "k23", "k31", "k32", "u"]

11-element Vector{String}:
 "x1"
 "x2"
 "x3"
 "k01"
 "k03"
 "k12"
 "k21"
 "k23"
 "k31"
 "k32"
 "u"

In [143]:
initial_conditions = rand(Float64, length(names))

11-element Vector{Float64}:
 0.2100312049265679
 0.9689761691345926
 0.8428562242281058
 0.7036813341915635
 0.11686689837900432
 0.9857097007742719
 0.48823818931402574
 0.8228000954290574
 0.46883394064184103
 0.9567535140551714
 0.22045313508867637

In [152]:
init_paper = [1,1,1, 0.022, 0.458, 0.22, 0.23, 0.16, 0.06, 0.32, 1 ]

11-element Vector{Float64}:
 1.0
 1.0
 1.0
 0.022
 0.458
 0.22
 0.23
 0.16
 0.06
 0.32
 1.0

In [153]:
find_numerical_transformations_hc(three_compartments, names, init_paper)

✓ # Computing specializations..    Time: 0:00:00
┌ Info: Computing normal forms (probabilistic)
│ Variables (7 in total): Nemo.QQMPolyRingElem[k01, k31, k21, k32, k12, k23, k03]
│ Up to degree: 2
└ Modulo: Finite field of characteristic 1073741827
[ Info: Used specialization points: 13
[ Info: Computing 8 Groebner bases for block orderings. Simplification code is 1
┌ Info: Final cleaning and simplification of generators. 
└ Out of 76 fractions 30 are syntactically unique.
[ Info: Checking inclusion with probability 0.995
[ Info: Inclusion checked in 0.0110344 seconds. Result: true
[ Info: Out of 7 initial generators there are 7 indepdendent
[ Info: The ranking of the new set of generators is 373398
[ Info: The search for identifiable functions concluded in 3.1817155 seconds


3-element Vector{PathResult}:
 PathResult:
 • return_code → :success
 • solution → ComplexF64[0.18412803278480558 - 1.2829270608442539e-48im, 0.281784413932063 + 1.5608945906938423e-48im, 0.06154698111514204 + 1.2107624136717646e-48im, 0.23 + 0.0im, 0.29975718895043274 - 1.2401628254827788e-48im, 0.06 + 0.0im, 0.3527833832175568 - 2.5090578715990487e-49im]
 • accuracy → 7.1929e-17
 • residual → 2.5562e-49
 • condition_jacobian → 7437.1
 • steps → 95 / 0
 • extended_precision → false
 • path_number → 1

 PathResult:
 • return_code → :success
 • solution → ComplexF64[0.6838719672152089 + 5.3809861030072976e-42im, 0.8830605460214387 + 1.0403239799147442e-41im, -0.7254195793638996 - 9.416725680262771e-42im, 0.23 + 0.0im, -0.17711698477562382 - 8.250845357944523e-42im, 0.06 + 0.0im, 0.5156040509028758 + 1.9954490131985395e-42im]
 • accuracy → 4.3881e-15
 • residual → 2.2204e-16
 • condition_jacobian → 809.27
 • steps → 84 / 0
 • extended_precision → false
 • path_number → 2

 PathResult:
 •

They are the same as the ones in the paper (taking only the real part, the complex part is negligible)

# Groebner basis initialization

In [7]:
using Groebner

In [ ]:
system = create

In [165]:
groebner(id_funct)

LoadError: MethodError: no method matching nvars(::AbstractAlgebra.Generic.FracField{Nemo.QQMPolyRingElem})

[0mClosest candidates are:
[0m  nvars([91m::AbstractAlgebra.Generic.UniversalPolyRing[39m)
[0m[90m   @[39m [32mAbstractAlgebra[39m [90mC:\Users\lgand\.julia\packages\AbstractAlgebra\zuBl1\src\generic\[39m[90m[4mUnivPoly.jl:41[24m[39m
[0m  nvars([91m::AbstractAlgebra.Generic.AbsMSeriesRing[39m)
[0m[90m   @[39m [32mAbstractAlgebra[39m [90mC:\Users\lgand\.julia\packages\AbstractAlgebra\zuBl1\src\generic\[39m[90m[4mAbsMSeries.jl:69[24m[39m
[0m  nvars([91m::AbstractAlgebra.Generic.LaurentMPolyWrapRing[39m)
[0m[90m   @[39m [32mAbstractAlgebra[39m [90mC:\Users\lgand\.julia\packages\AbstractAlgebra\zuBl1\src\generic\[39m[90m[4mLaurentMPoly.jl:26[24m[39m
[0m  ...


# Symbolic solver for three compartment problems 

In [92]:
id_funct = find_identifiable_functions(three_compartments)

✓ # Computing specializations..    Time: 0:00:02
┌ Info: Computing normal forms (probabilistic)
│ Variables (7 in total): Nemo.QQMPolyRingElem[k01, k31, k21, k32, k12, k23, k03]
│ Up to degree: 2
└ Modulo: Finite field of characteristic 1073741827
[ Info: Used specialization points: 13
[ Info: Computing 8 Groebner bases for block orderings. Simplification code is 1
┌ Info: Final cleaning and simplification of generators. 
└ Out of 76 fractions 30 are syntactically unique.
[ Info: Checking inclusion with probability 0.995
[ Info: Inclusion checked in 0.0178538 seconds. Result: true
[ Info: Out of 7 initial generators there are 7 indepdendent
[ Info: The ranking of the new set of generators is 373398
[ Info: The search for identifiable functions concluded in 4.8117884 seconds


7-element Vector{AbstractAlgebra.Generic.Frac{Nemo.QQMPolyRingElem}}:
 k21
 k31
 k31*k23 + k21*k23 + k21*k03
 k31*k32 + k31*k12 + k21*k32
 k01 + k32 + k12 + k23 + k03
 k01*k32 + k01*k12 + k01*k23 + k01*k03 + k31*k03 + k32*k03 + k12*k23 + k12*k03
 (k01*k31^2*k32 + k01*k31*k21*k32 - k31^2*k32^2 - k31^2*k32*k12 - 2*k31*k21*k32^2 - k31*k21*k32*k12 - k21^2*k32^2)//(k31^2*k21 - k31^2*k23 + k31*k21^2 + k31*k21*k32 + k31*k21*k12 - k31*k21*k23 - k31*k21*k03 + k21^2*k32)

In [156]:
factorized_id_funct = [AbstractAlgebra.factor(p) for p in id_funct]

7-element Vector{Fac{AbstractAlgebra.Generic.Frac{Nemo.QQMPolyRingElem}}}:
 k21
 k31
 k31*k23 + k21*k23 + k21*k03
 k31*k32 + k31*k12 + k21*k32
 k01 + k32 + k12 + k23 + k03
 k01*k32 + k01*k12 + k01*k23 + k01*k03 + k31*k03 + k32*k03 + k12*k23 + k12*k03
 (k01*k31^2*k32 + k01*k31*k21*k32 - k31^2*k32^2 - k31^2*k32*k12 - 2*k31*k21*k32^2 - k31*k21*k32*k12 - k21^2*k32^2)//(k31^2*k21 - k31^2*k23 + k31*k21^2 + k31*k21*k32 + k31*k21*k12 - k31*k21*k23 - k31*k21*k03 + k21^2*k32)

we cannot factorize now

In [154]:
x1, x2, x3, k01, k03, k12, k21, k23, k31, k32, u, y1, y2, y3, w01, w03, w12, w21, w23, w31, w32 = SymPy.symbols("x1 x2 x3 k01 k03 k12 k21 k23 k31 k32 u y1 y2 y3 w01 w03 w12 w21 w23 w31 w32")
system = create_system_of_equations(id_funct, [(k01, w01), (k03, w03), (k12, w12), (k21, w21), (k23, w23), (k31, w31), (k32, w32)])

7-element Vector{Sym}:
                                                                                                                                                                                                                                                                                                                                                                                                                                               k21 - w21
                                                                                                                                                                                                                                                                                                                                                                                                                                               k31 - w31
                                                                                               

In [160]:
for i in system
    println("old")
    println(i)
    println("new")
    println(SymPy.simplify(i))
end

old
k21 - w21
new
k21 - w21
old
k31 - w31
new
k31 - w31
old
k03*k21 + k21*k23 + k23*k31 - w03*w21 - w21*w23 - w23*w31
new
k03*k21 + k21*k23 + k23*k31 - w03*w21 - w21*w23 - w23*w31
old
k12*k31 + k21*k32 + k31*k32 - w12*w31 - w21*w32 - w31*w32
new
k12*k31 + k21*k32 + k31*k32 - w12*w31 - w21*w32 - w31*w32
old
k01 + k03 + k12 + k23 + k32 - w01 - w03 - w12 - w23 - w32
new
k01 + k03 + k12 + k23 + k32 - w01 - w03 - w12 - w23 - w32
old
k01*k03 + k01*k12 + k01*k23 + k01*k32 + k03*k12 + k03*k31 + k03*k32 + k12*k23 - w01*w03 - w01*w12 - w01*w23 - w01*w32 - w03*w12 - w03*w31 - w03*w32 - w12*w23
new
k01*k03 + k01*k12 + k01*k23 + k01*k32 + k03*k12 + k03*k31 + k03*k32 + k12*k23 - w01*w03 - w01*w12 - w01*w23 - w01*w32 - w03*w12 - w03*w31 - w03*w32 - w12*w23
old
(k01*k21*k31*k32 + k01*k31^2*k32 - k12*k21*k31*k32 - k12*k31^2*k32 - k21^2*k32^2 - 2*k21*k31*k32^2 - k31^2*k32^2)*(-w03*w21*w31 + w12*w21*w31 + w21^2*w31 + w21^2*w32 - w21*w23*w31 + w21*w31^2 + w21*w31*w32 - w23*w31^2) - (w01*w21*w31*w32 + w01*

In [176]:
for i in SymPy.solve(system[1:6], ( y1, y2, y3, w01, w03, w12, w21, w23, w31, w32))[1]
    println(i)
    println(simplify(i))
end

y1
y1
y2
y2
y3
y3
(k01*k31 + k03*k31 - k21*k31 - k21*k32 + k23*k31 - k31^2 - sqrt(k01^2*k31^2 - 2*k01*k03*k31^2 + 2*k01*k21*k31^2 + 2*k01*k21*k31*k32 - 2*k01*k23*k31^2 + 2*k01*k31^3 + k03^2*k31^2 - 2*k03*k21*k31^2 + 2*k03*k21*k31*k32 - 4*k03*k21*k31*w32 + 2*k03*k23*k31^2 - 2*k03*k31^3 - 4*k12*k21*k31*k32 + 4*k12*k21*k31*w32 + k21^2*k31^2 - 2*k21^2*k31*k32 + 4*k21^2*k31*w32 - 3*k21^2*k32^2 + 4*k21^2*k32*w32 - 2*k21*k23*k31^2 + 2*k21*k23*k31*k32 - 4*k21*k23*k31*w32 + 2*k21*k31^3 - 2*k21*k31^2*k32 + 4*k21*k31^2*w32 - 4*k21*k31*k32^2 + 4*k21*k31*k32*w32 + k23^2*k31^2 - 2*k23*k31^3 + 4*k23*k31^2*k32 - 4*k23*k31^2*w32 + k31^4))/(2*k31)
(k01*k31 + k03*k31 - k21*k31 - k21*k32 + k23*k31 - k31^2 - sqrt(k01^2*k31^2 - 2*k01*k03*k31^2 + 2*k01*k21*k31^2 + 2*k01*k21*k31*k32 - 2*k01*k23*k31^2 + 2*k01*k31^3 + k03^2*k31^2 - 2*k03*k21*k31^2 + 2*k03*k21*k31*k32 - 4*k03*k21*k31*w32 + 2*k03*k23*k31^2 - 2*k03*k31^3 - 4*k12*k21*k31*k32 + 4*k12*k21*k31*w32 + k21^2*k31^2 - 2*k21^2*k31*k32 + 4*k21^2*k31*w32 - 3*

## Groebner first

In [6]:
using Groebner

In [8]:
using DynamicPolynomials

@polyvar w01 w03 w12 w21 w23 w31 w32 k01 k03 k12 k21 k23 k31 k32 u 

(w01, w03, w12, w21, w23, w31, w32, k01, k03, k12, k21, k23, k31, k32, u)

In [9]:
F = [k21 - w21,
k31 - w31,
 k31*k23 + k21*k23 + k21*k03 - (w31*w23 + w21*w23 + w21*w03),
 k31*k32 + k31*k12 + k21*k32 - (w31*w32 + w31*w12 + w21*w32),
 k01 + k32 + k12 + k23 + k03 - (w01 + w32 + w12 + w23 + w03),
 k01*k32 + k01*k12 + k01*k23 + k01*k03 + k31*k03 + k32*k03 + k12*k23 + k12*k03 - (w01*w32 + w01*w12 + w01*w23 + w01*w03 + w31*w03 + w32*w03 + w12*w23 + w12*w03),
  (w31^2*w21 - w31^2*w23 + w31*w21^2 + w31*w21*w32 + w31*w21*w12 - w31*w21*w23 - w31*w21*w03 + w21^2*w32)*(k01*k31^2*k32 + k01*k31*k21*k32 - k31^2*k32^2 - k31^2*k32*k12 - 2*k31*k21*k32^2 - k31*k21*k32*k12 - k21^2*k32^2)- (w01*w31^2*w32 + w01*w31*w21*w32 - w31^2*w32^2 - w31^2*w32*w12 - 2*w31*w21*w32^2 - w31*w21*w32*w12 - w21^2*w32^2)*(k31^2*k21 - k31^2*k23 + k31*k21^2 + k31*k21*k32 + k31*k21*k12 - k31*k21*k23 - k31*k21*k03 + k21^2*k32)
     ]

7-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 k21 - w21
 k31 - w31
 k23k31 + k21k23 + k03k21 - w23w31 - w21w23 - w03w21
 k31k32 + k21k32 + k12k31 - w31w32 - w21w32 - w12w31
 k32 + k23 + k12 + k03 + k01 - w32 - w23 - w12 - w03 - w01
 k12k23 + k03k32 + k03k31 + k03k12 + k01k32 + k01k23 + k01k12 + k01k03 - w12w23 - w03w32 - w03w31 - w03w12 - w01w32 - w01w23 - w01w12 - w01w03
 -w31²w32²k23k31² + w31²w32²k21k31k32 + w31²w32²k21k31² - w31²w32²k21k23k31 + w31²w32²k21²k32 + w31²w32²k21²k31 + w31²w32²k12k21k31 - w31²w32²k03k21k31 + w23w31²k31²k32² + 2w23w31²k21k31k32² + w23w31²k21²k32² + w23w31²k12k31²k32 + w23w31²k12k21k31k32 - w23w31²k01k31²k32 - w23w31²k01k21k31k32 - w21w31w32k31²k32² - 2w21w31w32k21k31k32² - w21w31w32k21²k32² - w21w31w32k12k31²k32 - w21w31w32k12k21k31k32 + w21w31w32k01k31²k32 + w21w31w32k01k21k31k32 - 2w21w31w32²k23k31² + 2w21w31w32²k21k31k32 + 2w21w31w32²k21k31² - 2w21w31w32²k21k23k31 + 2w21w31w32²

In [26]:
for el in F
    println(simplify(el))
end

k21 - w21
k31 - w31
k23*k31 + k21*k23 + k03*k21 - w23*w31 - w21*w23 - w03*w21
k31*k32 + k21*k32 + k12*k31 - w31*w32 - w21*w32 - w12*w31
k32 + k23 + k12 + k03 + k01 - w32 - w23 - w12 - w03 - w01
k12*k23 + k03*k32 + k03*k31 + k03*k12 + k01*k32 + k01*k23 + k01*k12 + k01*k03 - w12*w23 - w03*w32 - w03*w31 - w03*w12 - w01*w32 - w01*w23 - w01*w12 - w01*w03
-w31^2*w32^2*k23*k31^2 + w31^2*w32^2*k21*k31*k32 + w31^2*w32^2*k21*k31^2 - w31^2*w32^2*k21*k23*k31 + w31^2*w32^2*k21^2*k32 + w31^2*w32^2*k21^2*k31 + w31^2*w32^2*k12*k21*k31 - w31^2*w32^2*k03*k21*k31 + w23*w31^2*k31^2*k32^2 + 2*w23*w31^2*k21*k31*k32^2 + w23*w31^2*k21^2*k32^2 + w23*w31^2*k12*k31^2*k32 + w23*w31^2*k12*k21*k31*k32 - w23*w31^2*k01*k31^2*k32 - w23*w31^2*k01*k21*k31*k32 - w21*w31*w32*k31^2*k32^2 - 2*w21*w31*w32*k21*k31*k32^2 - w21*w31*w32*k21^2*k32^2 - w21*w31*w32*k12*k31^2*k32 - w21*w31*w32*k12*k21*k31*k32 + w21*w31*w32*k01*k31^2*k32 + w21*w31*w32*k01*k21*k31*k32 - 2*w21*w31*w32^2*k23*k31^2 + 2*w21*w31*w32^2*k21*k31*k32 + 2*w21*w

In [10]:
groebner_F = groebner(F)

33-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 -k31 + w31
 -k21 + w21
 -k32 - k23 - k12 - k03 - k01 + w32 + w23 + w12 + w03 + w01
 -k31k32 - k21k32 - k12k31 + w32k31 + w32k21 + w12k31
 -k23k31 - k21k23 - k03k21 + w23k31 + w23k21 + w03k21
 k12k23 + k03k32 + k03k31 + k03k12 + k01k32 + k01k23 + k01k12 + k01k03 - w32k32 - w32k23 - w32k12 - w32k03 - w32k01 + w32² - w23k32 - w23k23 - w23k12 - w23k03 - w23k01 + 2w23w32 + w23² - w12k32 - w12k23 - w12k12 - w12k03 - w12k01 + 2w12w32 + w12w23 + w12² - w03k32 - w03k31 - w03k23 - w03k12 - w03k03 - w03k01 + w03w32 + 2w03w23 + w03w12 + w03²
 k23k31² + k21k23k31 + k01k23k31 - k01k21k32 - k01k12k21 + w32k21k32 + w32k21k23 + w32k12k21 + w32k01k21 - w32²k21 - w23k31² - w23k23k31 + w23k21k32 - w23k21k31 - w23k01k31 - 2w23w32k21 + w23²k31 + w12k21k32 + w12k12k21 + w12k01k21 - 2w12w32k21 - w12²k21 - w03k23k31 + w03w23k31
 k23²k31² - k21k23k31² + k21k23²k31 - k21²k23k31 + k03k21k23k3

In [27]:
for el in groebner_F
    println(simplify(el))
end

-k31 + w31
-k21 + w21
-k32 - k23 - k12 - k03 - k01 + w32 + w23 + w12 + w03 + w01
-k31*k32 - k21*k32 - k12*k31 + w32*k31 + w32*k21 + w12*k31
-k23*k31 - k21*k23 - k03*k21 + w23*k31 + w23*k21 + w03*k21
k12*k23 + k03*k32 + k03*k31 + k03*k12 + k01*k32 + k01*k23 + k01*k12 + k01*k03 - w32*k32 - w32*k23 - w32*k12 - w32*k03 - w32*k01 + w32^2 - w23*k32 - w23*k23 - w23*k12 - w23*k03 - w23*k01 + 2*w23*w32 + w23^2 - w12*k32 - w12*k23 - w12*k12 - w12*k03 - w12*k01 + 2*w12*w32 + w12*w23 + w12^2 - w03*k32 - w03*k31 - w03*k23 - w03*k12 - w03*k03 - w03*k01 + w03*w32 + 2*w03*w23 + w03*w12 + w03^2
k23*k31^2 + k21*k23*k31 + k01*k23*k31 - k01*k21*k32 - k01*k12*k21 + w32*k21*k32 + w32*k21*k23 + w32*k12*k21 + w32*k01*k21 - w32^2*k21 - w23*k31^2 - w23*k23*k31 + w23*k21*k32 - w23*k21*k31 - w23*k01*k31 - 2*w23*w32*k21 + w23^2*k31 + w12*k21*k32 + w12*k12*k21 + w12*k01*k21 - 2*w12*w32*k21 - w12^2*k21 - w03*k23*k31 + w03*w23*k31
k23^2*k31^2 - k21*k23*k31^2 + k21*k23^2*k31 - k21^2*k23*k31 + k03*k21*k23*k31 - k01*k21

-k21*k23*k31^3*k32^3 - k21*k23^2*k31^3*k32^2 + k21^2*k31^2*k32^4 + k21^2*k31^3*k32^3 - 3*k21^2*k23*k31^2*k32^3 + k21^2*k23*k31^3*k32^2 - k21^2*k23^2*k31^2*k32^2 + 3*k21^3*k31*k32^4 + 3*k21^3*k31^2*k32^3 - 3*k21^3*k23*k31*k32^3 + k21^3*k23*k31^2*k32^2 + 2*k21^4*k32^4 + 3*k21^4*k31*k32^3 - k21^4*k23*k32^3 + k21^5*k32^3 - k12*k21*k23*k31^3*k32^2 + 2*k12*k21^2*k31^2*k32^3 + k12*k21^2*k31^3*k32^2 - 2*k12*k21^2*k23*k31^2*k32^2 + 4*k12*k21^3*k31*k32^3 + 2*k12*k21^3*k31^2*k32^2 - k12*k21^3*k23*k31*k32^2 + k12*k21^4*k32^3 + k12*k21^4*k31*k32^2 + k12^2*k21^2*k31^2*k32^2 + k12^2*k21^3*k31*k32^2 - k03*k21^2*k31^2*k32^3 - k03*k21^2*k23*k31^2*k32^2 - 2*k03*k21^3*k31*k32^3 - k03*k21^4*k32^3 - k03*k12*k21^2*k31^2*k32^2 - k03*k12*k21^3*k31*k32^2 + k01*k21*k23*k31^3*k32^2 - k01*k21^2*k31^2*k32^3 - k01*k21^2*k31^3*k32^2 + 3*k01*k21^2*k23*k31^2*k32^2 - 2*k01*k21^3*k31*k32^3 - 2*k01*k21^3*k31^2*k32^2 + k01*k21^3*k23*k31*k32^2 - k01*k21^4*k31*k32^2 - k01*k12*k21^2*k31^2*k32^2 - k01*k12*k21^3*k31*k32^2 + k01

2*k23*k31^4*k32^3 + 2*k23*k31^5*k32^2 - 3*k23^2*k31^4*k32^2 - k21*k31^3*k32^4 - 2*k21*k31^4*k32^3 - k21*k31^5*k32^2 + 6*k21*k23*k31^3*k32^3 + 8*k21*k23*k31^4*k32^2 - 7*k21*k23^2*k31^3*k32^2 - 2*k21^2*k31^2*k32^4 - 6*k21^2*k31^3*k32^3 - 4*k21^2*k31^4*k32^2 + 6*k21^2*k23*k31^2*k32^3 + 13*k21^2*k23*k31^3*k32^2 - 5*k21^2*k23^2*k31^2*k32^2 - 6*k21^3*k31^2*k32^3 - 6*k21^3*k31^3*k32^2 + 2*k21^3*k23*k31*k32^3 + 9*k21^3*k23*k31^2*k32^2 - k21^3*k23^2*k31*k32^2 + k21^4*k32^4 - 2*k21^4*k31*k32^3 - 4*k21^4*k31^2*k32^2 + 2*k21^4*k23*k31*k32^2 - k21^5*k31*k32^2 + 4*k12*k23*k31^4*k32^2 + 2*k12*k23*k31^5*k32 - 2*k12*k23^2*k31^4*k32 - 3*k12*k21*k31^3*k32^3 - 4*k12*k21*k31^4*k32^2 - k12*k21*k31^5*k32 + 10*k12*k21*k23*k31^3*k32^2 + 6*k12*k21*k23*k31^4*k32 - 3*k12*k21*k23^2*k31^3*k32 - 5*k12*k21^2*k31^2*k32^3 - 10*k12*k21^2*k31^3*k32^2 - 3*k12*k21^2*k31^4*k32 + 8*k12*k21^2*k23*k31^2*k32^2 + 6*k12*k21^2*k23*k31^3*k32 - k12*k21^2*k23^2*k31^2*k32 - k12*k21^3*k31*k32^3 - 8*k12*k21^3*k31^2*k32^2 - 3*k12*k21^3*k

*k32 - w23^2*k21*k31^3*k32^2 - w03*k23*k31^4*k32^2 - w03*k12*k23*k31^4*k32 + w03*k01*k23*k31^4*k32
k23*k31^4*k32^4 - k21*k31^3*k32^5 - k21*k31^4*k32^4 + 4*k21*k23*k31^3*k32^4 + k21*k23^2*k31^3*k32^3 - 4*k21^2*k31^2*k32^5 - 4*k21^2*k31^3*k32^4 + 6*k21^2*k23*k31^2*k32^4 - k21^2*k23*k31^3*k32^3 + k21^2*k23^2*k31^2*k32^3 - 6*k21^3*k31*k32^5 - 6*k21^3*k31^2*k32^4 + 4*k21^3*k23*k31*k32^4 - k21^3*k23*k31^2*k32^3 - 3*k21^4*k32^5 - 4*k21^4*k31*k32^4 + k21^4*k23*k32^4 - k21^5*k32^4 + 2*k12*k23*k31^4*k32^3 - 3*k12*k21*k31^3*k32^4 - 2*k12*k21*k31^4*k32^3 + 6*k12*k21*k23*k31^3*k32^3 - 9*k12*k21^2*k31^2*k32^4 - 6*k12*k21^2*k31^3*k32^3 + 6*k12*k21^2*k23*k31^2*k32^3 - 9*k12*k21^3*k31*k32^4 - 6*k12*k21^3*k31^2*k32^3 + 2*k12*k21^3*k23*k31*k32^3 - 2*k12*k21^4*k32^4 - 2*k12*k21^4*k31*k32^3 + k12^2*k23*k31^4*k32^2 - 3*k12^2*k21*k31^3*k32^3 - k12^2*k21*k31^4*k32^2 + 2*k12^2*k21*k23*k31^3*k32^2 - 6*k12^2*k21^2*k31^2*k32^3 - 2*k12^2*k21^2*k31^3*k32^2 + k12^2*k21^2*k23*k31^2*k32^2 - 3*k12^2*k21^3*k31*k32^3 - k

k23^2*k31^5*k32^2 - 4*k21*k23*k31^4*k32^3 - 2*k21*k23*k31^5*k32^2 + 4*k21*k23^2*k31^4*k32^2 + 3*k21^2*k31^3*k32^4 + 4*k21^2*k31^4*k32^3 + k21^2*k31^5*k32^2 - 12*k21^2*k23*k31^3*k32^3 - 8*k21^2*k23*k31^4*k32^2 + 7*k21^2*k23^2*k31^3*k32^2 + 8*k21^3*k31^2*k32^4 + 12*k21^3*k31^3*k32^3 + 4*k21^3*k31^4*k32^2 - 12*k21^3*k23*k31^2*k32^3 - 13*k21^3*k23*k31^3*k32^2 + 5*k21^3*k23^2*k31^2*k32^2 + 6*k21^4*k31*k32^4 + 12*k21^4*k31^2*k32^3 + 6*k21^4*k31^3*k32^2 - 4*k21^4*k23*k31*k32^3 - 9*k21^4*k23*k31^2*k32^2 + k21^4*k23^2*k31*k32^2 + k21^5*k32^4 + 4*k21^5*k31*k32^3 + 4*k21^5*k31^2*k32^2 - 2*k21^5*k23*k31*k32^2 + k21^6*k31*k32^2 + k12*k23^2*k31^5*k32 - 6*k12*k21*k23*k31^4*k32^2 - 2*k12*k21*k23*k31^5*k32 + 3*k12*k21*k23^2*k31^4*k32 + 7*k12*k21^2*k31^3*k32^3 + 6*k12*k21^2*k31^4*k32^2 + k12*k21^2*k31^5*k32 - 14*k12*k21^2*k23*k31^3*k32^2 - 6*k12*k21^2*k23*k31^4*k32 + 3*k12*k21^2*k23^2*k31^3*k32 + 13*k12*k21^3*k31^2*k32^3 + 14*k12*k21^3*k31^3*k32^2 + 3*k12*k21^3*k31^4*k32 - 10*k12*k21^3*k23*k31^2*k32^2 -

...

2*k23*k31^4*k32^6 - 8*k23*k31^5*k32^5 - 2*k21*k31^3*k32^7 + 6*k21*k31^4*k32^6 + 8*k21*k31^5*k32^5 + 6*k21*k23*k31^3*k32^6 - 32*k21*k23*k31^4*k32^5 - 6*k21^2*k31^2*k32^7 + 26*k21^2*k31^3*k32^6 + 32*k21^2*k31^4*k32^5 + 6*k21^2*k23*k31^2*k32^6 - 48*k21^2*k23*k31^3*k32^5 - 6*k21^3*k31*k32^7 + 42*k21^3*k31^2*k32^6 + 48*k21^3*k31^3*k32^5 + 2*k21^3*k23*k31*k32^6 - 32*k21^3*k23*k31^2*k32^5 - 2*k21^4*k32^7 + 30*k21^4*k31*k32^6 + 32*k21^4*k31^2*k32^5 - 8*k21^4*k23*k31*k32^5 + 8*k21^5*k32^6 + 8*k21^5*k31*k32^5 + 5*k12*k23*k31^4*k32^5 - 19*k12*k23*k31^5*k32^4 - k12*k23^2*k31^4*k32^4 - 7*k12*k21*k31^3*k32^6 + 22*k12*k21*k31^4*k32^5 + 19*k12*k21*k31^5*k32^4 + 14*k12*k21*k23*k31^3*k32^5 - 58*k12*k21*k23*k31^4*k32^4 - 3*k12*k21*k23^2*k31^3*k32^4 - 17*k12*k21^2*k31^2*k32^6 + 70*k12*k21^2*k31^3*k32^5 + 59*k12*k21^2*k31^4*k32^4 + 14*k12*k21^2*k23*k31^2*k32^5 - 60*k12*k21^2*k23*k31^3*k32^4 - 3*k12*k21^2*k23^2*k31^2*k32^4 - 13*k12*k21^3*k31*k32^6 + 76*k12*k21^3*k31^2*k32^5 + 63*k12*k21^3*k31^3*k32^4 + 6*k1

4*k12*k21^2*k23*k31^3 - 7*w32^4*k12*k21^2*k23^2*k31^2 - 8*w32^4*k12*k21^3*k31*k32^2 + 6*w32^4*k12*k21^3*k31^2*k32 + 15*w32^4*k12*k21^3*k31^3 + 10*w32^4*k12*k21^3*k23*k31*k32 + 4*w32^4*k12*k21^3*k23*k31^2 - 3*w32^4*k12*k21^3*k23^2*k31 - 3*w32^4*k12*k21^4*k32^2 + 3*w32^4*k12*k21^4*k31^2 + 2*w32^4*k12*k21^4*k23*k32 + 6*w32^4*k12*k21^4*k23*k31 - 2*w32^4*k12*k21^5*k32 - 3*w32^4*k12*k21^5*k31 + 2*w32^4*k12^2*k21*k23*k31^3 - 4*w32^4*k12^2*k21^2*k31^2*k32 + 3*w32^4*k12^2*k21^2*k31^3 + 5*w32^4*k12^2*k21^2*k23*k31^2 - 5*w32^4*k12^2*k21^3*k31*k32 + 3*w32^4*k12^2*k21^3*k23*k31 - w32^4*k12^2*k21^4*k32 - 3*w32^4*k12^2*k21^4*k31 - w32^4*k12^3*k21^2*k31^2 - w32^4*k12^3*k21^3*k31 + w32^4*k03*k23^2*k31^4 - 6*w32^4*k03*k21*k23*k31^3*k32 + 8*w32^4*k03*k21*k23*k31^4 + 5*w32^4*k03*k21*k23^2*k31^3 + 5*w32^4*k03*k21^2*k31^2*k32^2 - 4*w32^4*k03*k21^2*k31^3*k32 - 9*w32^4*k03*k21^2*k31^4 - 14*w32^4*k03*k21^2*k23*k31^2*k32 + 10*w32^4*k03*k21^2*k23*k31^3 + 7*w32^4*k03*k21^2*k23^2*k31^2 + 8*w32^4*k03*k21^3*k31*k32^

Excessive output truncated after 524290 bytes.

*k32^3 + 36*k12*k21^2*k23*k31^2*k32^5 - 176*k12*k21^2*k23*k31^3*k32^4 - 43

LoadError: InterruptException:

In [29]:
a = (k31 + w32)^2 + k31

k31 + k31² + 2w32k31 + w32²

In [33]:
AbstractAlgebra.factor(a)

LoadError: function factor is not implemented for argument
Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}: k31 + k31^2 + 2*w32*k31 + w32^2


In [34]:
_, (p, n, d, q) = PolynomialRing(QQ, ["p","n","d","q"], ordering=:deglex)

F = [p^5 - n, p^10 - d, p^25 - q]   # initial constraints

G = groebner(F)   # more nice and useful constraints

4-element Vector{AbstractAlgebra.Generic.MPoly{Rational{BigInt}}}:
 n^2 - d
 d^3 - n*q
 n*d^2 - q
 p^5 - n

In [35]:
_, (x, y) = QQ["x", "y"]
F = [x^3 + y^2, x*y + x^2]

2-element Vector{AbstractAlgebra.Generic.MPoly{Rational{BigInt}}}:
 x^3 + y^2
 x^2 + x*y

In [12]:
_, ( w01, w03, w12, w21, w23, w31, w32, k01, k03, k12, k21, k23, k31, k32, u ) = PolynomialRing(QQ, ["w01", "w03", "w12", "w21", "w23", "w31", "w32", "k01", "k03", "k12", "k21", "k23", "k31", "k32", "u" ], ordering=:lex)


(Multivariate polynomial ring in 15 variables over rationals, AbstractAlgebra.Generic.MPoly{Rational{BigInt}}[w01, w03, w12, w21, w23, w31, w32, k01, k03, k12, k21, k23, k31, k32, u])

In [13]:
F = [k21 - w21,
k31 - w31,
 k31*k23 + k21*k23 + k21*k03 - (w31*w23 + w21*w23 + w21*w03),
 k31*k32 + k31*k12 + k21*k32 - (w31*w32 + w31*w12 + w21*w32),
 k01 + k32 + k12 + k23 + k03 - (w01 + w32 + w12 + w23 + w03),
 k01*k32 + k01*k12 + k01*k23 + k01*k03 + k31*k03 + k32*k03 + k12*k23 + k12*k03 - (w01*w32 + w01*w12 + w01*w23 + w01*w03 + w31*w03 + w32*w03 + w12*w23 + w12*w03),
  (w31^2*w21 - w31^2*w23 + w31*w21^2 + w31*w21*w32 + w31*w21*w12 - w31*w21*w23 - w31*w21*w03 + w21^2*w32)*(k01*k31^2*k32 + k01*k31*k21*k32 - k31^2*k32^2 - k31^2*k32*k12 - 2*k31*k21*k32^2 - k31*k21*k32*k12 - k21^2*k32^2)- (w01*w31^2*w32 + w01*w31*w21*w32 - w31^2*w32^2 - w31^2*w32*w12 - 2*w31*w21*w32^2 - w31*w21*w32*w12 - w21^2*w32^2)*(k31^2*k21 - k31^2*k23 + k31*k21^2 + k31*k21*k32 + k31*k21*k12 - k31*k21*k23 - k31*k21*k03 + k21^2*k32)
     ]

7-element Vector{AbstractAlgebra.Generic.MPoly{Rational{BigInt}}}:
 -w21 + k21
 -w31 + k31
 -w03*w21 - w21*w23 - w23*w31 + k03*k21 + k21*k23 + k23*k31
 -w12*w31 - w21*w32 - w31*w32 + k12*k31 + k21*k32 + k31*k32
 -w01 - w03 - w12 - w23 - w32 + k01 + k03 + k12 + k23 + k32
 -w01*w03 - w01*w12 - w01*w23 - w01*w32 - w03*w12 - w03*w31 - w03*w32 - w12*w23 + k01*k03 + k01*k12 + k01*k23 + k01*k32 + k03*k12 + k03*k31 + k03*k32 + k12*k23
 w01*w21*w31*w32*k03*k21*k31 - w01*w21*w31*w32*k12*k21*k31 - w01*w21*w31*w32*k21^2*k31 - w01*w21*w31*w32*k21^2*k32 + w01*w21*w31*w32*k21*k23*k31 - w01*w21*w31*w32*k21*k31^2 - w01*w21*w31*w32*k21*k31*k32 + w01*w21*w31*w32*k23*k31^2 + w01*w31^2*w32*k03*k21*k31 - w01*w31^2*w32*k12*k21*k31 - w01*w31^2*w32*k21^2*k31 - w01*w31^2*w32*k21^2*k32 + w01*w31^2*w32*k21*k23*k31 - w01*w31^2*w32*k21*k31^2 - w01*w31^2*w32*k21*k31*k32 + w01*w31^2*w32*k23*k31^2 - w03*w21*w31*k01*k21*k31*k32 - w03*w21*w31*k01*k31^2*k32 + w03*w21*w31*k12*k21*k31*k32 + w03*w21*w31*k12*k31^2*k32 + w03*

In [30]:
typeof(k21 - w21)

AbstractAlgebra.Generic.MPoly{Rational{BigInt}}

In [14]:
G = groebner(F, ordering = Lex([w01, w03, w12, w21, w23, w31, w32, k01, k03, k12, k21, k23, k31, k32, u]))

33-element Vector{AbstractAlgebra.Generic.MPoly{Rational{BigInt}}}:
 w32^3*k03^2*k21^3*k31^2 + w32^3*k03^2*k21^2*k31^3 - 2*w32^3*k03*k12*k21^3*k31^2 - 2*w32^3*k03*k12*k21^2*k31^3 - 2*w32^3*k03*k21^4*k31^2 - 2*w32^3*k03*k21^4*k31*k32 + 2*w32^3*k03*k21^3*k23*k31^2 - 4*w32^3*k03*k21^3*k31^3 - 4*w32^3*k03*k21^3*k31^2*k32 + 4*w32^3*k03*k21^2*k23*k31^3 - 2*w32^3*k03*k21^2*k31^4 - 2*w32^3*k03*k21^2*k31^3*k32 + 2*w32^3*k03*k21*k23*k31^4 + w32^3*k12^2*k21^3*k31^2 + w32^3*k12^2*k21^2*k31^3 + 2*w32^3*k12*k21^4*k31^2 + 2*w32^3*k12*k21^4*k31*k32 - 2*w32^3*k12*k21^3*k23*k31^2 + 4*w32^3*k12*k21^3*k31^3 + 4*w32^3*k12*k21^3*k31^2*k32 - 4*w32^3*k12*k21^2*k23*k31^3 + 2*w32^3*k12*k21^2*k31^4 + 2*w32^3*k12*k21^2*k31^3*k32 - 2*w32^3*k12*k21*k23*k31^4 + w32^3*k21^5*k31^2 + 2*w32^3*k21^5*k31*k32 + w32^3*k21^5*k32^2 - 2*w32^3*k21^4*k23*k31^2 - 2*w32^3*k21^4*k23*k31*k32 + 3*w32^3*k21^4*k31^3 + 6*w32^3*k21^4*k31^2*k32 + 3*w32^3*k21^4*k31*k32^2 + w32^3*k21^3*k23^2*k31^2 - 6*w32^3*k21^3*k23*k31^3 - 6*w32^3*k21^3*k

In [15]:
G[end]

w01 + w03 + w12 + w23 + w32 - k01 - k03 - k12 - k23 - k32

In [12]:
AbstractAlgebra.factor(G[1])

LoadError: function factor is not implemented for argument
AbstractAlgebra.Generic.MPoly{Rational{BigInt}}: w32^3*k03^2*k21^3*k31^2 + w32^3*k03^2*k21^2*k31^3 - 2*w32^3*k03*k12*k21^3*k31^2 - 2*w32^3*k03*k12*k21^2*k31^3 - 2*w32^3*k03*k21^4*k31^2 - 2*w32^3*k03*k21^4*k31*k32 + 2*w32^3*k03*k21^3*k23*k31^2 - 4*w32^3*k03*k21^3*k31^3 - 4*w32^3*k03*k21^3*k31^2*k32 + 4*w32^3*k03*k21^2*k23*k31^3 - 2*w32^3*k03*k21^2*k31^4 - 2*w32^3*k03*k21^2*k31^3*k32 + 2*w32^3*k03*k21*k23*k31^4 + w32^3*k12^2*k21^3*k31^2 + w32^3*k12^2*k21^2*k31^3 + 2*w32^3*k12*k21^4*k31^2 + 2*w32^3*k12*k21^4*k31*k32 - 2*w32^3*k12*k21^3*k23*k31^2 + 4*w32^3*k12*k21^3*k31^3 + 4*w32^3*k12*k21^3*k31^2*k32 - 4*w32^3*k12*k21^2*k23*k31^3 + 2*w32^3*k12*k21^2*k31^4 + 2*w32^3*k12*k21^2*k31^3*k32 - 2*w32^3*k12*k21*k23*k31^4 + w32^3*k21^5*k31^2 + 2*w32^3*k21^5*k31*k32 + w32^3*k21^5*k32^2 - 2*w32^3*k21^4*k23*k31^2 - 2*w32^3*k21^4*k23*k31*k32 + 3*w32^3*k21^4*k31^3 + 6*w32^3*k21^4*k31^2*k32 + 3*w32^3*k21^4*k31*k32^2 + w32^3*k21^3*k23^2*k31^2 - 6*

In [13]:
AbstractAlgebra.deflation(G[1])

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [40]:
AbstractAlgebra.deflation(G[2])

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [41]:
AbstractAlgebra.leading_coefficient(G[1])

1//1

In [42]:
AbstractAlgebra.trailing_coefficient(G[1])

1//1

In [43]:
AbstractAlgebra.leading_monomial(G[1])

w32^3*k03^2*k21^3*k31^2

In [44]:
AbstractAlgebra.leading_term(G[1])

w32^3*k03^2*k21^3*k31^2

In [46]:
AbstractAlgebra.leading_term(AbstractAlgebra.tail(G[1]))

w32^3*k03^2*k21^2*k31^3

let's try...

In [16]:
Gstring = []
for el in G
    push!(Gstring, string(el))
end


In [17]:
k01, k03, k12, k21, k23, k31, k32, u, w01, w03, w12, w21, w23, w31, w32 = SymPy.symbols("k01 k03 k12 k21 k23 k31 k32 u w01 w03 w12 w21 w23 w31 w32")



(k01, k03, k12, k21, k23, k31, k32, u, w01, w03, w12, w21, w23, w31, w32)

In [18]:
SN = []
for el in Gstring
    push!(SN, sympify(el))
end

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\lgand\.julia\packages\PyCall\ilqDX\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'RecursionError'>
RecursionError('maximum recursion depth exceeded during compilation')
  File "C:\Users\lgand\.julia\conda\3\x86_64\lib\site-packages\sympy\core\sympify.py", line 495, in sympify
    expr = parse_expr(a, local_dict=locals, transformations=transformations, evaluate=evaluate)
  File "C:\Users\lgand\.julia\conda\3\x86_64\lib\site-packages\sympy\parsing\sympy_parser.py", line 1087, in parse_expr
    raise e from ValueError(f"Error from parse_expr with transformed code: {code!r}")
  File "C:\Users\lgand\.julia\conda\3\x86_64\lib\site-packages\sympy\parsing\sympy_parser.py", line 1078, in parse_expr
    rv = eval_expr(code, local_dict, global_dict)
  File "C:\Users\lgand\.julia\conda\3\x86_64\lib\site-packages\sympy\parsing\sympy_parser.py", line 906, in eval_expr
    expr = eval(


In [19]:
SN

19-element Vector{Any}:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [18]:
g1s = sympify(Gstring[1])

   2        2    3    2      2        2    3              2            4    2 
k01 *k03*k21 *k31 *k32  - k01 *k03*k21 *k31 *k32*w32 + k01 *k03*k21*k31 *k32  

     2            4              2        2    3    2      2        2    3    
- k01 *k03*k21*k31 *k32*w32 - k01 *k12*k21 *k31 *k32  + k01 *k12*k21 *k31 *k32

          2            4    2      2            4              2    3    3    
*w32 - k01 *k12*k21*k31 *k32  + k01 *k12*k21*k31 *k32*w32 - k01 *k21 *k31 *k32

2      2    3    3              2    3    2    3      2    3    2    2        
  + k01 *k21 *k31 *k32*w32 - k01 *k21 *k31 *k32  + k01 *k21 *k31 *k32 *w32 + k

  2    2        3    2      2    2        3                2    2    4    2   
01 *k21 *k23*k31 *k32  - k01 *k21 *k23*k31 *k32*w32 - 2*k01 *k21 *k31 *k32  + 

     2    2    4                2    2    3    3        2    2    3    2      
2*k01 *k21 *k31 *k32*w32 - 2*k01 *k21 *k31 *k32  + 2*k01 *k21 *k31 *k32 *w32 +

      2            4    2        2            

In [23]:
S1 = SymPy.factor(sympify(SN[1]))

                         /                               2          2         
(k21 + k31)*(-k32 + w32)*\k03*k21*k31 - k12*k21*k31 - k21 *k31 - k21 *k32 + k2

                   2                        2\ /     2    2                  2
1*k23*k31 - k21*k31  - k21*k31*k32 + k23*k31 /*\- k01 *k31 *k32 + k01*k03*k31 

                    2                  2                  2                   
*w32 + 2*k01*k12*k31 *k32 - k01*k12*k31 *w32 - k01*k21*k31 *w32 + 2*k01*k21*k3

     2                                      2              3                2 
1*k32  - 2*k01*k21*k31*k32*w32 + k01*k23*k31 *w32 - k01*k31 *w32 + 2*k01*k31 *

   2          2                      2                                        
k32  - k01*k31 *k32*w32 - k03*k12*k31 *w32 - k03*k21*k31*k32*w32 + k03*k21*k31

    2          2              2    2          2    2                  2       
*w32  - k03*k31 *k32*w32 - k12 *k31 *k32 + k12 *k31 *w32 + k12*k21*k31 *w32 - 

                 2                            

In [22]:
S[1]

                         /                               2          2         
(k21 + k31)*(-k32 + w32)*\k03*k21*k31 - k12*k21*k31 - k21 *k31 - k21 *k32 + k2

                   2                        2\ /     2    2                  2
1*k23*k31 - k21*k31  - k21*k31*k32 + k23*k31 /*\- k01 *k31 *k32 + k01*k03*k31 

                    2                  2                  2                   
*w32 + 2*k01*k12*k31 *k32 - k01*k12*k31 *w32 - k01*k21*k31 *w32 + 2*k01*k21*k3

     2                                      2              3                2 
1*k32  - 2*k01*k21*k31*k32*w32 + k01*k23*k31 *w32 - k01*k31 *w32 + 2*k01*k31 *

   2          2                      2                                        
k32  - k01*k31 *k32*w32 - k03*k12*k31 *w32 - k03*k21*k31*k32*w32 + k03*k21*k31

    2          2              2    2          2    2                  2       
*w32  - k03*k31 *k32*w32 - k12 *k31 *k32 + k12 *k31 *w32 + k12*k21*k31 *w32 - 

                 2                            

In [23]:
S[2]

-k31 + w31

In [28]:
S[3]

            /                               2          2                      
(k21 + k31)*\k03*k21*k31 - k12*k21*k31 - k21 *k31 - k21 *k32 + k21*k23*k31 - k

      2                        2\ /                 2                      2  
21*k31  - k21*k31*k32 + k23*k31 /*\- k01*k03*k21*k31 *k32 + k01*k03*k21*k31 *w

                    2                      2              2    2              
32 + k01*k12*k21*k31 *k32 - k01*k12*k21*k31 *w32 + k01*k21 *k31 *k32 - k01*k21

2    2              2        2          2                              2      
 *k31 *w32 + k01*k21 *k31*k32  - k01*k21 *k31*k32*w32 - k01*k21*k23*k31 *k32 +

                2                  3                  3                  2    
 k01*k21*k23*k31 *w32 + k01*k21*k31 *k32 - k01*k21*k31 *w32 + k01*k21*k31 *k32

2              2                      3              3                        
  - k01*k21*k31 *k32*w32 - k01*k23*k31 *k32 + k01*k31 *k32*w23 + k03*k12*k21*k

  2                      2              2     

In [25]:
S[4]

            /                               2          2                      
(k21 + k31)*\k03*k21*k31 - k12*k21*k31 - k21 *k31 - k21 *k32 + k21*k23*k31 - k

      2                        2\ /                                           
21*k31  - k21*k31*k32 + k23*k31 /*\-k01*k21*k31*k32 + k01*k21*k31*w32 + k12*k2

                                 2    2        2              2    2          
1*k31*k32 - k12*k21*k31*w32 + k21 *k32  - 2*k21 *k32*w32 + k21 *w32  + k21*k31

    2                            2          2        \
*k32  - k21*k31*k32*w32 - k23*k31 *w32 + k31 *w23*w32/

In [26]:
S[5]

            /                               2          2                      
(k21 + k31)*\k03*k21*k31 - k12*k21*k31 - k21 *k31 - k21 *k32 + k21*k23*k31 - k

      2                        2\ /   2            2      2                   
21*k31  - k21*k31*k32 + k23*k31 /*\k01 *k21*k31*k32  - k01 *k21*k31*k32*w32 - 

   2    2    2      2    2                              2                     
k01 *k31 *k32  + k01 *k31 *k32*w32 + k01*k03*k21*k31*k32  - 2*k01*k03*k21*k31*

                             2              2                      2    2     
k32*w32 + k01*k03*k21*k31*w32  + k01*k03*k31 *k32*w32 - k01*k03*k31 *w32  - 3*

                   2                                                  2       
k01*k12*k21*k31*k32  + 4*k01*k12*k21*k31*k32*w32 - k01*k12*k21*k31*w32  + 2*k0

         2    2                2                      2    2          2       
1*k12*k31 *k32  - 3*k01*k12*k31 *k32*w32 + k01*k12*k31 *w32  - k01*k21 *k31*k3

 2            2                      2        

In [24]:
F = [k21 - w21,
k31 - w31,
 k31*k23 + k21*k23 + k21*k03 - (w31*w23 + w21*w23 + w21*w03),
 k31*k32 + k31*k12 + k21*k32 - (w31*w32 + w31*w12 + w21*w32),
 k01 + k32 + k12 + k23 + k03 - (w01 + w32 + w12 + w23 + w03),
 k01*k32 + k01*k12 + k01*k23 + k01*k03 + k31*k03 + k32*k03 + k12*k23 + k12*k03 - (w01*w32 + w01*w12 + w01*w23 + w01*w03 + w31*w03 + w32*w03 + w12*w23 + w12*w03),
  (w31^2*w21 - w31^2*w23 + w31*w21^2 + w31*w21*w32 + w31*w21*w12 - w31*w21*w23 - w31*w21*w03 + w21^2*w32)*(k01*k31^2*k32 + k01*k31*k21*k32 - k31^2*k32^2 - k31^2*k32*k12 - 2*k31*k21*k32^2 - k31*k21*k32*k12 - k21^2*k32^2)- (w01*w31^2*w32 + w01*w31*w21*w32 - w31^2*w32^2 - w31^2*w32*w12 - 2*w31*w21*w32^2 - w31*w21*w32*w12 - w21^2*w32^2)*(k31^2*k21 - k31^2*k23 + k31*k21^2 + k31*k21*k32 + k31*k21*k12 - k31*k21*k23 - k31*k21*k03 + k21^2*k32)
     ]

7-element Vector{Sym}:
                                                                                                                                                                                                                                                                                                                                                                                                                                               k21 - w21
                                                                                                                                                                                                                                                                                                                                                                                                                                               k31 - w31
                                                                                               

In [25]:
F1 = [-k32 + w32,
    k21 - w21,
k31 - w31,
 k31*k23 + k21*k23 + k21*k03 - (w31*w23 + w21*w23 + w21*w03),
 k31*k32 + k31*k12 + k21*k32 - (w31*w32 + w31*w12 + w21*w32),
 k01 + k32 + k12 + k23 + k03 - (w01 + w32 + w12 + w23 + w03),
 k01*k32 + k01*k12 + k01*k23 + k01*k03 + k31*k03 + k32*k03 + k12*k23 + k12*k03 - (w01*w32 + w01*w12 + w01*w23 + w01*w03 + w31*w03 + w32*w03 + w12*w23 + w12*w03),
  (w31^2*w21 - w31^2*w23 + w31*w21^2 + w31*w21*w32 + w31*w21*w12 - w31*w21*w23 - w31*w21*w03 + w21^2*w32)*(k01*k31^2*k32 + k01*k31*k21*k32 - k31^2*k32^2 - k31^2*k32*k12 - 2*k31*k21*k32^2 - k31*k21*k32*k12 - k21^2*k32^2)- (w01*w31^2*w32 + w01*w31*w21*w32 - w31^2*w32^2 - w31^2*w32*w12 - 2*w31*w21*w32^2 - w31*w21*w32*w12 - w21^2*w32^2)*(k31^2*k21 - k31^2*k23 + k31*k21^2 + k31*k21*k32 + k31*k21*k12 - k31*k21*k23 - k31*k21*k03 + k21^2*k32)
     ]

8-element Vector{Sym}:
                                                                                                                                                                                                                                                                                                                                                                                                                                              -k32 + w32
                                                                                                                                                                                                                                                                                                                                                                                                                                               k21 - w21
                                                                                               

In [29]:
typeof(F)

Vector{Sym} (alias for Array{Sym, 1})

In [26]:
sol1 = SymPy.solve(F1, (w01, w03, w12, w21, w23, w31, w32))

1-element Vector{NTuple{7, Sym}}:
 (k01, k03, k12, k21, k23, k31, k32)

ok, trivial solution. Let's try with the second

In [27]:
F2 = [k03 * k21 * k31 − k12 * k21*k31 − k21^2 * k31 − k21^2 * k32+k21*k23* k31−k21*k31^2−k21*k31*k32+k23*k31^2,
    k21 - w21,
k31 - w31,
 k31*k23 + k21*k23 + k21*k03 - (w31*w23 + w21*w23 + w21*w03),
 k31*k32 + k31*k12 + k21*k32 - (w31*w32 + w31*w12 + w21*w32),
 k01 + k32 + k12 + k23 + k03 - (w01 + w32 + w12 + w23 + w03),
 k01*k32 + k01*k12 + k01*k23 + k01*k03 + k31*k03 + k32*k03 + k12*k23 + k12*k03 - (w01*w32 + w01*w12 + w01*w23 + w01*w03 + w31*w03 + w32*w03 + w12*w23 + w12*w03),
  (w31^2*w21 - w31^2*w23 + w31*w21^2 + w31*w21*w32 + w31*w21*w12 - w31*w21*w23 - w31*w21*w03 + w21^2*w32)*(k01*k31^2*k32 + k01*k31*k21*k32 - k31^2*k32^2 - k31^2*k32*k12 - 2*k31*k21*k32^2 - k31*k21*k32*k12 - k21^2*k32^2)- (w01*w31^2*w32 + w01*w31*w21*w32 - w31^2*w32^2 - w31^2*w32*w12 - 2*w31*w21*w32^2 - w31*w21*w32*w12 - w21^2*w32^2)*(k31^2*k21 - k31^2*k23 + k31*k21^2 + k31*k21*k32 + k31*k21*k12 - k31*k21*k23 - k31*k21*k03 + k21^2*k32)
     ]

8-element Vector{Sym}:
                                                                                                                                                                                                                                                                                                                                                   k03*k21*k31 - k12*k21*k31 - k21^2*k31 - k21^2*k32 + k21*k23*k31 - k21*k31^2 - k21*k31*k32 + k23*k31^2
                                                                                                                                                                                                                                                                                                                                                                                                                                               k21 - w21
                                                                                               

In [28]:
sol2 = SymPy.solve(F2, (w01, w03, w12, w21, w23, w31, w32))

InterruptException: InterruptException: